In [8]:
import tensorflow as tf
from keras.utils import Sequence
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
import matplotlib.pyplot as plt
import os
import pandas as pd

In [9]:
# hyper parameters
batch_size = 32

img_width, img_height = 384, 288

color_mode, color_channels = 'grayscale', 1

dataset = os.getcwd().replace('\DeepDetection', '\DataSet')

In [10]:
train_path = '{}/train'.format(dataset)
valid_path = '{}/val'.format(dataset)
test_path = '{}/test'.format(dataset)

print('train:%d, valid:%d, test:%d' %(len(os.listdir(train_path)), len(os.listdir(valid_path)), len(os.listdir(test_path))))

train:2550, valid:1275, test:1276


In [11]:
data = pd.read_csv('{}/labels.csv'.format(dataset))
data.head()

,frame,x1,y1,x2,y2,x3,y3,x4,y4,x5,...,x6,y6,x7,y7,x8,y8,x9,y9,x10,y10
0,1,47,112,86,162,147,198,218,201,265,...,304,121,234,104,181,101,132,100,82,107
1,2,46,113,80,157,130,190,207,203,264,...,298,124,258,109,209,106,145,103,88,108
2,3,51,116,88,162,129,187,188,204,261,...,300,126,248,108,185,104,136,102,90,107
3,4,48,114,74,152,110,176,170,201,243,...,302,127,259,108,200,102,152,100,91,105
4,5,51,111,87,158,147,191,229,193,275,...,302,123,262,103,206,94,138,91,92,95


In [12]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import math
import random

def load_image(im):
    return img_to_array(load_img(im, grayscale=True)) / 255.

class DataSequence(Sequence):
    """
    Keras Sequence object to train a model on larger-than-memory data.
    """
    def __init__(self, df, data_path, batch_size, mode='train'):
        self.df = df
        self.bsz = batch_size
        self.mode = mode

        # Take labels and a list of image locations in memory
        self.labels = self.df['label'].values
        self.im_list = self.df['image_name'].apply(lambda x: os.path.join(data_path, x)).tolist()

    def __len__(self):
        return int(math.ceil(len(self.df) / float(self.bsz)))

    def on_epoch_end(self):
        # Shuffles indexes after each epoch if in training mode
        self.indexes = range(len(self.im_list))
        if self.mode == 'train':
            self.indexes = random.sample(self.indexes, k=len(self.indexes))

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return self.labels[idx * self.bsz: (idx + 1) * self.bsz]

    def get_batch_features(self, idx):
        # Fetch a batch of inputs
        return np.array([load_image(im) for im in self.im_list[idx * self.bsz: (1 + idx) * self.bsz]])

    def __getitem__(self, idx):
        batch_x = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_x, batch_y

In [14]:
train_gen = train_datagen.flow_from_directory(train_path, 
                                              target_size=(img_width, img_height),
                                              class_mode='categorical', 
                                              batch_size=batch_size, 
                                              color_mode=color_mode)

valid_gen = val_test_datagen.flow_from_directory(valid_path, 
                                                 target_size=(img_width, img_height),
                                                 class_mode='categorical', 
                                                 batch_size=batch_size,
                                                 color_mode=color_mode)

test_gen = val_test_datagen.flow_from_directory(test_path, 
                                                target_size=(img_width, img_height),
                                                class_mode='categorical', 
                                                batch_size=batch_size, 
                                                color_mode=color_mode)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
